In [1]:
import gurobipy as gp
import numpy as np
import pandas as pd
from scipy import stats


from gurobi_implementation import GurobiSolution
from data_market import Seller, MarketOperator, Market, Buyer
from p2p_gurobi import Agents, FirstStageMarket

In [2]:
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

In [3]:
connection_matrix = [[0, 1, 1, 1, 1],
                    [1, 0, 1, 1, 1],
                    [1, 1, 0, 1, 1],
                    [1, 1, 1, 0, 1],
                    [1, 1, 1, 1, 0]]

kappa = [[0, 10, 10, 10, 10],
        [10, 0, 10, 10, 10],
        [10, 10, 0, 10, 10],
        [10, 10, 10, 0, 10],
        [10, 10, 10, 10, 0]]                   

In [4]:
#model_1
actual_generations = [3, 4, 5, 4, 3]
demand = [6, 7, 8, 9, 10]

price_da_buy = 3
price_da_sell = 2
price_rt_buy = 4
price_rt_sell = 1

base_forecasts = [stats.norm(loc = generation, scale = 0.5) for generation in actual_generations]

agents_forecasts = []
for base_forecast in base_forecasts:
    probas, values = np.histogram(base_forecasts[0].rvs(100000), bins = 100, density = True)
    probas = probas / probas.sum()
    agents_forecasts.append((probas, values))

agents = []
for id, actual_generation in enumerate(actual_generations):
    agent = Agents(id = id,
                probabilities = agents_forecasts[id][0],
                generation_values = agents_forecasts[id][1],
                demand = demand[id],
                connections = connection_matrix[id],
                kappa = kappa[id])

    agents.append(agent)

In [5]:
model_1 = gp.Model()

In [6]:
market_1 = FirstStageMarket(agents= agents,
                            model = model_1,
                            price_da_buy = price_da_buy,
                            price_da_sell = price_da_sell,
                            price_rt_buy = price_rt_buy,
                            price_rt_sell = price_rt_sell)

In [7]:
market_1.build_model()

In [8]:
model_1.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 525 rows, 1035 columns and 2565 nonzeros
Model fingerprint: 0xbe41884f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 3e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [8e-01, 9e+00]
Presolve removed 40 rows and 60 columns
Presolve time: 0.00s
Presolved: 485 rows, 975 columns, 2415 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     694    7.8052491e+01   0.000000e+00   0.000000e+00      0s

Solved in 694 iterations and 0.01 seconds (0.01 work units)
Optimal objective  7.805249091e+01


In [9]:
def second_stage_resolution(model, agents, actual_generations):
    second_stage_decisions = []
    actual_costs = []
    for agent in agents:
        second_stage_decision = (agent.demand 
                                - actual_generations[agent.id] 
                                - model.getVarByName(f'Agent {agent.id} day-ahead purchase').X
                                + model.getVarByName(f'Agent {agent.id} day-ahead sale').X
                                - model.getVarByName(f'Agent {agent.id} net trading').X)

        second_stage_cost = price_rt_buy * second_stage_decision if second_stage_decision > 0 else price_rt_sell * second_stage_decision

        second_stage_decisions.append(second_stage_decision)

        actual_cost = (price_da_buy * model.getVarByName(f'Agent {agent.id} day-ahead purchase').X 
                    - price_da_sell * model.getVarByName(f'Agent {agent.id} day-ahead sale').X
                    + second_stage_cost )


        actual_costs.append(actual_cost)

    return second_stage_decisions, actual_costs
        

In [10]:
second_stage_resolution(model_1, agents, actual_generations)

([0.20897845221904987,
  -0.7983248101316471,
  -1.792530266070413,
  -0.8133998688502144,
  0.19959543400563362],
 [72.82295698535897,
  -0.7983248101316471,
  -1.792530266070413,
  -0.8133998688502144,
  0.7983817360225345])

In [11]:
#model_2
actual_generations = [3, 4, 5, 4, 3]
demand = [6, 7, 8, 9, 10]

price_da_buy = 3
price_da_sell = 2
price_rt_buy = 4
price_rt_sell = 1

base_forecasts = [stats.norm(loc = generation, scale = 1.0) for generation in actual_generations]

agents_forecasts = []
for base_forecast in base_forecasts:
    probas, values = np.histogram(base_forecasts[0].rvs(100000), bins = 100, density = True)
    probas = probas / probas.sum()
    agents_forecasts.append((probas, values))

agents = []
for id, actual_generation in enumerate(actual_generations):
    agent = Agents(id = id,
                probabilities = agents_forecasts[id][0],
                generation_values = agents_forecasts[id][1],
                demand = demand[id],
                connections = connection_matrix[id],
                kappa = kappa[id])

    agents.append(agent)

In [12]:
model_2 = gp.Model()

market_2 = FirstStageMarket(agents= agents,
                            model = model_2,
                            price_da_buy = price_da_buy,
                            price_da_sell = price_da_sell,
                            price_rt_buy = price_rt_buy,
                            price_rt_sell = price_rt_sell)

In [13]:
market_2.build_model()

In [14]:
model_2.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 525 rows, 1035 columns and 2565 nonzeros
Model fingerprint: 0x8b96c8b5
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 3e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [3e-02, 1e+01]
Presolve removed 41 rows and 62 columns
Presolve time: 0.00s
Presolved: 484 rows, 973 columns, 2410 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     692    8.1072963e+01   0.000000e+00   0.000000e+00      0s

Solved in 692 iterations and 0.01 seconds (0.01 work units)
Optimal objective  8.107296292e+01


In [15]:
second_stage_resolution(model_2, agents, actual_generations)

([0.20897845221904987,
  -0.7983248101316471,
  -1.792530266070413,
  -0.8133998688502144,
  0.19959543400563362],
 [72.82295698535897,
  -0.7983248101316471,
  -1.792530266070413,
  -0.8133998688502144,
  0.7983817360225345])